In [121]:
%matplotlib inline
import sys
sys.path.append("/Users/yasser.zidani/qolmat/")
import numpy as np
import pandas as pd
import sklearn.base
import sklearn.utils

from qolmat.imputations import preprocessing, imputers
from qolmat.imputations.imputers import ImputerRegressor
from qolmat.benchmark import missing_patterns
from qolmat.benchmark import comparator
from qolmat.utils import data
from sklearn.pipeline import Pipeline

## Single imputation using MCA: 

In [55]:
from scipy import sparse
from sklearn.utils import extmath
import dataclasses

try:
    import fbpca

    FBPCA_INSTALLED = True
except ImportError:
    FBPCA_INSTALLED = False 
    
class SVD:
    def __init__(self, U, s, V):
        self.U = U
        self.s = s
        self.V = V
        
def compute_svd(X, 
                n_components, 
                n_iter, 
                random_state, 
                engine) -> SVD:
    
    
    """Computes an SVD with k components."""

    # TODO: support sample weights

    # Compute the SVD
    if engine == "fbpca":
        if FBPCA_INSTALLED:
            U, s, V = fbpca.pca(X, k=n_components, n_iter=n_iter)
        else:
            raise ValueError("fbpca is not installed; please install it if you want to use it")
    elif engine == "scipy":
        U, s, V = scipy.linalg.svd(X)
        U = U[:, :n_components]
        s = s[:n_components]
        V = V[:n_components, :]
    elif engine == "sklearn":
        U, s, V = extmath.randomized_svd(
            X, n_components=n_components, n_iter=n_iter, random_state=random_state
        )
    else:
        raise ValueError("engine has to be one of ('fbpca', 'scipy', 'sklearn')")

    # U, V = extmath.svd_flip(U, V)
    return SVD(U, s, V)

In [227]:
from scipy.linalg import svd
from numpy.linalg import norm


def create_disjunctive_table(X):
    """
    Converts categorical data to a disjunctive table (one-hot encoded) and retains NaN values
    in such a way that if the original value is NaN, all dummy columns for that category are NaN.
    """
    dummies = pd.get_dummies(X, dummy_na=False, dtype=int)

    # Loop through original columns to adjust the one-hot encoded results
    for col in X.columns:
        # Identify rows where the original column is NaN
        na_rows = X[col].isna()

        # List the new dummy columns for this specific original column
        new_cols = [new_col for new_col in dummies.columns if new_col.startswith(col + '_')]

        # Set all dummy columns to NaN where the original value was NaN
        dummies.loc[na_rows, new_cols] = np.nan
    
    return dummies

def initialize_missing_values(Z):
    """Initializes missing values using the proportion of observed categories."""
    
    proportions = Z.mean()  # Proportions for each column
    Z_filled = Z.fillna(proportions)
    return Z_filled


def compute_gsvd(X, D_sigma, R, p, n_components):
    """
    Perform a Generalized Singular Value Decomposition (GSVD) on the matrix X,
    taking into account the diagonal matrix D_sigma for column weights and matrix R for row weights.
    
    Parameters:
    - X: Data matrix (indicator matrix of dummy variables)
    - D_sigma: Diagonal matrix based on category proportions
    - R: Row weighting matrix, often the identity matrix scaled appropriately
    - p: Total number of observations or another normalization factor
    - n_components: Number of singular vectors and values to retain

    Returns:
    - U, s, Vt: Reduced singular vectors and values after GSVD
    """
    # Compute the inverse of D_sigma scaled by 1/p
    D_sigma_inv_scaled = (1/p) * np.linalg.inv(D_sigma)

    # Apply column weights by multiplying X with D_sigma_inv_scaled
    X_weighted = X.dot(D_sigma_inv_scaled)

    # Apply row weights by pre-multiplying by R (if R is not the identity matrix)
    if not np.array_equal(R, np.eye(X.shape[0])):
        X_weighted = R.dot(X_weighted)

    # Perform SVD on the weighted matrix
    U, s, Vt = svd(X_weighted, full_matrices=False)

    # Return the components specified by n_components
    return U[:, :n_components], s[:n_components], Vt[:n_components, :]



def update_D_sigma(Z):
    """ Update the diagonal matrix D_sigma based on the proportions of observed categories. """
    
    category_proportions = Z.sum(axis=0) / Z.sum().sum()
    return np.diag(1 / (category_proportions + 1e-6))  # Avoid division by zero

class MIMCA:
    def __init__(self, n_components=2, max_iter=10, tolerance=1e-6):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tolerance = tolerance

    def fit(self, X):
        Z = create_disjunctive_table(X)
        Z = initialize_missing_values(Z)
        # we start by initialisation 
        M0 = Z.mean(axis=0)
        D_sigma = update_D_sigma(Z)

        for _ in range(self.max_iter):
            Z_old = Z.copy()
            
            #TODO : insert so,e asserts overhere for dimensions checking 
            U, s, Vt = compute_gsvd(Z - M0, D_sigma, R=None, p=Z.shape[0], n_components=self.n_components)
            
            #pour avoir une matrice de dimensions (j,j)
            Lambda_half = np.sqrt(np.diag(s))
            
            Z_fitted = np.dot(np.dot(U, Lambda_half), Vt) + M0  # Apply the reconstruction formula
            
            W = ~(np.isnan(Z_old).astype(bool))  # 1 for observed, 0 for missing
            Z_new = W * Z_old + (1 - W) * Z_fitted  # Blend using Hadamard product
            if np.linalg.norm(Z_new - Z_old, 'fro') < self.tolerance:
                break
            
            ###########################################################################################
            # we can as well go with hadamard product : 
            #U, s, Vt = compute_gsvd(Z - M0, D_sigma, None, Z.shape[0], self.n_components)
            #S = np.diag(s[:self.n_components])  # Truncate to the first S dimensions
            #Z_fitted = np.dot(np.dot(U, S), Vt) + M0
            # Compute the weighting matrix W
            #W = np.isnan(Z_old).astype(int)
            #W[W == 1] = 0
            #W[W == 0] = 1
            # Impute missing values
            #Z_new = W * Z_old + (1 - W) * Z_fitted
            #M0 = Z.mean(axis=0)
            ###########################################################################################

            Z = Z_new
            D_sigma = update_D_sigma(Z)  # Update D_sigma based on new Z
        


        self.imputed_data_ = Z
        return self

    def transform(self):
        """Returns the imputed data after fitting the model."""
        return self.imputed_data_

## Data Manipulation : 

In [178]:
from qolmat.utils import data
df = data.get_data("Titanic")
df.insert(0, 'ID', df.index)
df = df.drop(columns=["Survived", "Fare", "Parch", "SibSp", "Age"])
df

,ID,Sex,Embarked
0,0,male,S
1,1,female,C
2,2,female,S
3,3,female,S
4,4,male,S
...,...,...,...
886,886,male,S
887,887,female,S
888,888,female,S
889,889,male,C


In [179]:
df.columns

Index(['ID', 'Sex', 'Embarked'], dtype='object')

In [191]:
from qolmat.imputations import preprocessing, imputers
cols_to_impute = ['Sex', 'Embarked']
ratio_masked = 0.1
generator_holes = missing_patterns.UniformHoleGenerator(
    n_splits=2,
    subset=cols_to_impute,
    ratio_masked=ratio_masked,
    sample_proportional=False,
)
generator_holes.fit(df)
masks = generator_holes.split(df)
for mask in masks:
    masked_df = df.mask(mask)
df=masked_df
#df[df.Embarked.isna()]

/Users/yasser.zidani/qolmat/qolmat/benchmark/missing_patterns.py:202: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_mask[col].iloc[indices] = True


In [213]:
df

,ID,Sex,Embarked
0,0,male,S
1,1,female,C
2,2,female,S
3,3,NaN,S
4,4,NaN,S
...,...,...,...
886,886,male,S
887,887,female,NaN
888,888,NaN,S
889,889,NaN,C


In [225]:
def create_disjunctive_table(X):
    """
    Converts categorical data to a disjunctive table (one-hot encoded) and retains NaN values
    in such a way that if the original value is NaN, all dummy columns for that category are NaN.
    """
    dummies = pd.get_dummies(X, dummy_na=False, dtype=int)

    # Loop through original columns to adjust the one-hot encoded results
    for col in X.columns:
        # Identify rows where the original column is NaN
        na_rows = X[col].isna()

        # List the new dummy columns for this specific original column
        new_cols = [new_col for new_col in dummies.columns if new_col.startswith(col + '_')]

        # Set all dummy columns to NaN where the original value was NaN
        dummies.loc[na_rows, new_cols] = np.nan
    
    return dummies
Z= create_disjunctive_table(df)
#Z.loc[Z['ID']==888]

In [226]:
Z

,ID,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0.0,1.0,0.0,0.0,1.0
1,1,1.0,0.0,1.0,0.0,0.0
2,2,1.0,0.0,0.0,0.0,1.0
3,3,NaN,NaN,0.0,0.0,1.0
4,4,NaN,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...
886,886,0.0,1.0,0.0,0.0,1.0
887,887,1.0,0.0,NaN,NaN,NaN
888,888,NaN,NaN,0.0,0.0,1.0
889,889,NaN,NaN,1.0,0.0,0.0


In [232]:
M_0= initialize_missing_values(Z)
M_0

,ID,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0.000000,1.000000,0.00000,0.000000,1.000000
1,1,1.000000,0.000000,1.00000,0.000000,0.000000
2,2,1.000000,0.000000,0.00000,0.000000,1.000000
3,3,0.355837,0.644163,0.00000,0.000000,1.000000
4,4,0.355837,0.644163,0.00000,0.000000,1.000000
...,...,...,...,...,...,...
886,886,0.000000,1.000000,0.00000,0.000000,1.000000
887,887,1.000000,0.000000,0.19323,0.088858,0.717913
888,888,0.355837,0.644163,0.00000,0.000000,1.000000
889,889,0.355837,0.644163,1.00000,0.000000,0.000000


In [231]:
D0_sigma= update_D_sigma(Z)
D0_sigma

array([[1.00358037e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.57031679e+03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.68055868e+02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.89607743e+03,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.27646824e+03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 7.81147681e+02]])

In [235]:
import numpy as np
from scipy import linalg
rng = np.random.default_rng()
m, n = 9, 6
a = rng.standard_normal((m, n)) + 1.j*rng.standard_normal((m, n))
U, s, Vh = linalg.svd(a)
s.shape

(6,)

In [244]:
# Reconstruct SVD
from numpy import array
from numpy import diag
from numpy import dot
from scipy.linalg import svd
A = array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)
# Singular-value decomposition
U, s, VT = svd(A)
# create n x n Sigma matrix
Sigma = diag(s)
# reconstruct matrix
B = U.dot(Sigma.dot(VT))
print(B)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [242]:
B = U.dot(Sigma.dot(Vh))
print(B)

ValueError: shapes (9,9) and (6,6) not aligned: 9 (dim 1) != 6 (dim 0)